In [1]:
import pandas as pd 
import numpy as np
import scipy.stats as st
from scipy.stats import chi2_contingency

from itertools import combinations
from statsmodels.sandbox.stats.multicomp import multipletests

from scipy.stats import fisher_exact

import statsmodels.stats.contingency_tables as contingency

from scipy.stats import binom_test

from scipy.stats import wilcoxon

from scipy.stats import mannwhitneyu

from scipy.stats import kruskal
import pingouin as pg
from scikit_posthocs import posthoc_dunn

import scikit_posthocs as scipost

## 1) Teste $\chi^2$ 

### 1.1) Teste de Homogeneidade

**Teste de diferença de proporções para K populações independentes.**

A administração de um hospital deseja verificar se luvas de três marcas (A, B e C) são homogêneas quanto à permeabilidade a vírus. Para isto, realizou um experimento, no qual 240 luvas da marca A, 240 luvas da marca B e 300 luvas da marca C foram submetidas à tensão.

Durante os testes, 151 luvas da marca A (62.9%), 134 luvas da marca B (55.8%) e 177 luvas da marca C (59.0%) deixaram passar vírus. Os dados do experimento apresentam evidências estatísticas suficientes contra a hipótese de que as três marcas possuem a mesma permeabilidade?

$$ H_0: p_1 = p_2 = p_3\\H_1: \text{Pelo menos uma é diferente} $$

In [2]:
df_luvas = pd.read_csv('Permeabilidade_Luvas.csv', sep= ',')
df_luvas.head()

,Luva,Resposta
0,A,Sim
1,A,Sim
2,A,Sim
3,A,Sim
4,A,Sim


#### 1.1.1) Contruindo a tabela de contigência 

In [3]:
tabela_cont = pd.crosstab(df_luvas['Luva'], df_luvas['Resposta'], margins=True)
tabela_cont

Resposta,Nao,Sim,All
Luva,,,
A,89,151,240
B,106,134,240
C,123,177,300
All,318,462,780


In [4]:
tabela_cont = pd.crosstab(df_luvas['Luva'], df_luvas['Resposta'], margins=False)
tabela_cont

Resposta,Nao,Sim
Luva,,
A,89,151
B,106,134
C,123,177


#### TH

In [5]:
alpha = 0.05

chi2_obs, p, gl, exp = chi2_contingency(tabela_cont, correction=True)

chi_critic = st.chi2.ppf(1 - alpha, gl) # unilateral 

if(abs(chi2_obs) > chi_critic):
    print(f"Rejeitamos a hipótese nula.")
else:
    print(f"Não temos evidências para rejeitar a hipótese nula.")

print(f"Valor-p: {p:.4f}")

Não temos evidências para rejeitar a hipótese nula.
Valor-p: 0.2859


#### Teste post-hoc - em caso de rejeição de Ho 

In [6]:
df = tabela_cont

all_combinations = list(combinations(df.index, 2))
p_vals = []

for comb in all_combinations:
    new_df = df[(df.index == comb[0]) | (df.index == comb[1])]
    chi2, p, gl, ex = st.chi2_contingency(new_df, correction=True)
    p_vals.append(p)
    
reject_list, corrected_p_vals = multipletests(p_vals, method='fdr_bh')[:2]

print("original p-value\tcorrected p-value\treject?")
for p_val, corr_p_val, reject, comb in zip(p_vals, corrected_p_vals, reject_list, all_combinations):
        print(f"{comb}: p_value: {p_val:3f}; corrected: {corr_p_val:3f} reject: {reject}") 

original p-value	corrected p-value	reject?
('A', 'B'): p_value: 0.137024; corrected: 0.411072 reject: False
('A', 'C'): p_value: 0.402332; corrected: 0.514226 reject: False
('B', 'C'): p_value: 0.514226; corrected: 0.514226 reject: False


## 2) Teste exato de Fischer 

**Teste de diferença de proporções para 2 populações independentes.**

Em um estudo de situações nas quais pessoas ameaçam suicido saltando de um local alto, foi observado que ocorrem vaias e provocações por certos espectadores em alguns casos e outros não. 

Um estudo examinou 21 relatos de suicídios e investigou a relação entre provocações por multidões e o mês do ano (indicador grosseiro para temperatura). A hipótese é de que há um aumento de provocações por multidões quando faz calor. 

Usando a α=0.05 verifique a hipótese. 

|                | Provocação | Não provocação | Total |
|:--------------:|:----------:|:--------------:|:-----:|
| Junho-Setembro |      8     |        4       |   12  |
|  Outubro-Maio  |      2     |        7       |   9   |
|      Total     |     10     |       11       |   21  |

$$ H_0: p_1 = p_2 \\H_1: \text{Pelo menos uma é diferente} $$

In [7]:
alpha = 0.05

contingency_table = [[8, 4], [2, 7]]

estat_obs, p_valor = fisher_exact(contingency_table, alternative='greater')

if p_valor < alpha:
    print("Rejeitamos a hipótese nula")
else:
    print("Não rejeitamos a hipótese nula")
    
print("Valor p:", p_valor.round(3))

Não rejeitamos a hipótese nula
Valor p: 0.056


## 3) Teste de McNemar

**Teste de diferença de proporções para 2 populações dependentes.**

Necessita de tabela de contingência 2x2

Numa campanha política, após uma série de fatos que supostamente difamaram a imagem do candidato B em favor do candidato A, investigou-se as mudanças ocorridas com relação à preferência do eleitorado. 

O que podemos concluir com os dados observados a um nível de 5%?

|             | Candidato A  | Candidato B |
|:-----------:|:------------:|:-----------:|
| Candidato A |      83      |      47     |
| Candidato B |      18      |      52     |

$$ H_0: p_1 = p_2 \\H_1: p_1 \neq p_2 $$

In [8]:
alpha = 0.05

contingency_table = np.array([[83, 47], [18, 52]]) 

result = contingency.mcnemar(contingency_table, exact=False, correction=True)
# exact=True --> binomial distribution.
# exact=False --> chisquare distribution.

if result.pvalue < alpha:
    print("Rejeitamos a hipótese nula.")
    if contingency_table[1, 0] + contingency_table[1, 1] < 10:
        print("Nota: A suposição b + c >= 10 não é atendida.")
else:
    print("Não temos evidências para rejeitar a hipótese nula.")

print("Valor p:", result.pvalue.round(3))

Rejeitamos a hipótese nula.
Valor p: 0.001


## 4) Teste Binomial 

**Teste de proporção para uma amostra**

Usado quando $n\le35$ (pelo TLC não temos distribuição normal para usar o teste Z) 

Dois supermercados disputam a preferência dos consumidores de uma cidade. O supermercado A realiza então uma campanha com distribuição de prêmios para aumentar o número de fregueses. 100 consumidores foram acompanhados antes e depois da campanha, onde perguntou-se a cada um deles sobre a sua preferência. Os resultados seguem abaixo. Utilize α=0,05.

|       |  A |  B | Total |
|:-----:|:--:|:--:|:-----:|
|   A   | 37 |  3 |   40  |
|   B   | 13 | 47 |   60  |
| Total | 50 | 50 |  100  |

In [9]:
alpha = 0.05

n = 16
x = 3
p0 = 0.5

p_valor  = binom_test(x=x, n=n, p=p0, alternative='less')

if p_valor < alpha:
    print("Rejeitamos a hipótese nula.")
else:
    print("Não rejeitamos a hipótese nula.")
    
print("Valor p:", p_valor.round(3))

Rejeitamos a hipótese nula.
Valor p: 0.011


/var/folders/y9/p4kzsp2s2m97hj45x03kq3g00000gn/T/ipykernel_3363/1801530233.py:7: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_valor  = binom_test(x=x, n=n, p=p0, alternative='less')


Uma empresa lançou recentemente no mercado uma dieta de emagrecimento. E pretende-se averiguar se a proporção de consumidores com idades superiores a 60 anos é maior que 0.2. 

Para tal procedeu na escolha aleatória de 20 indivíduos entre os adeptos da nova dieta, e apurou que
apenas 5 tinham idade superior a 60 anos. Utilize α=0,05.

In [10]:
alpha = 0.05

n = 20
x = 5
p0 = 0.2

p_valor  = binom_test(x=x, n=n, p=p0, alternative='greater')

if p_valor < alpha:
    print("Rejeitamos a hipótese nula.")
else:
    print("Não rejeitamos a hipótese nula.")
    
print("Valor p:", p_valor.round(3))

Não rejeitamos a hipótese nula.
Valor p: 0.37


/var/folders/y9/p4kzsp2s2m97hj45x03kq3g00000gn/T/ipykernel_3363/2070244784.py:7: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_valor  = binom_test(x=x, n=n, p=p0, alternative='greater')


## 5) Teste da Mediana 

**Teste para diferença de medianas de duas populações independentes** 

Se (n1+n2) > 20 --> Teste $\chi^2$ + correção de Yates 

Se (n1+n2) $\le$ 20 --> Teste exato de Fisher 

Os dados apresentam a quantidade de salários mínimos dos funcionários de uma empresa de dois setores: Recursos Humanos e Controle de Qualidade.  Verifique se os salários do RH provém de populações com mediana maior do que da população de CQ. 

| RH | CQ |
|----|----|
| 4  | 11 |
| 3  | 10 |
| 8  | 7  |
| 3  | 6  |
| 5  | 5  |
| 7  | 8  |
| 2  | 9  |
|    | 10 |

$$ H_0: md_{rh} = md_{cq} \\H_1: md_{rh} > md_{cq} $$

In [11]:
alpha = 0.05

contingency_table = [[1, 5], [6, 3]]

estat_obs, p_valor = fisher_exact(contingency_table, alternative='greater')

if p_valor < alpha:
    print("Rejeitamos a hipótese nula")
else:
    print("Não rejeitamos a hipótese nula")
    
print("Valor p:", p_valor.round(3))

Não rejeitamos a hipótese nula
Valor p: 0.994


Nos mesmos dois setores da empresa citada anteriormente, registrou-se os escores de um teste de inteligência emocional. Verifique a hipótese de que as amostras provêm de populações com a mesma mediana. 

| RH | CQ |
|----|----|
| 15 | 8  |
| 12 | 7  |
| 13 | 14 |
| 9  | 10 |
| 18 | 8  |
| 11 | 6  |
| 12 | 7  |
| 13 | 6  |
| 14 | 8  |
| 14 | 4  |
| 12 |    |
| 12 |    |
| 10 |    |
| 11 |    |

In [12]:
alpha = 0.05

contingency_table = [[10, 1], [4, 9]]

chi2_obs, p, gl, Exp = chi2_contingency(contingency_table)

chi_critic = st.chi2.ppf(1 - alpha, gl)

if(abs(chi2_obs) > chi_critic):
    print(f"Rejeitamos a hipótese nula.")
else:
    print(f"Não temos evidências para rejeitar a hipótese nula.")

print(f"Valor-p: {p:.4f}")

Rejeitamos a hipótese nula.
Valor-p: 0.0104


## 6) Teste de Postos com Sinais de Wilcoxon

**Teste para diferença de medianas de duas populações dependentes**

Os dados a seguir são das colheitas de espigas de milho (em libras por acre) de dois diferentes tipos de sementes (normais e secas no forno) que foram usados em lotes adjacentes. Usando o teste dos wilcoxom, verifique se há diferença entre as colheitas de milho de semente normal e semente secada no forno. 
Utilizar α=0,05.

| Normal | Seca |
|:------:|:------:|
|  1903  |  2009  |
|  1935  |  1915  |
|  1910  |  2011  |
| 2496   | 2463   |
| 2108   | 2180   |
| 1961   | 1925   |
| 2060   | 2122   |
| 1444   | 1482   |
| 1612   | 1542   |
| 1316   | 1443   |
| 1511   | 1535   |

In [13]:
normal = [1903, 1935, 1910, 2496, 2108, 1961, 2060, 1444, 1612, 1316, 1511]
seca = [2009, 1915, 2011, 2463, 2180, 1925, 2122, 1482, 1542, 1443, 1535]

In [14]:
alpha = 0.05

estat_obs, p_valor = wilcoxon(normal, seca, zero_method='wilcox', 
                         correction=False, alternative='two-sided', method = 'auto')

if p_valor < alpha:
    print("Rejeitamos a hipótese nula")
else:
    print("Não rejeitamos a hipótese nula")

print(f"Valor-p: {p_valor:.4f}")

Não rejeitamos a hipótese nula
Valor-p: 0.1230


#### No caso de uma amostra 

In [15]:
alpha = 0.05

amostra = [10.5, 7.5, 18.5, 14, 9, 0, 15.5, 17, 20, 1.5, 1.5, 3, 13, 0, 23, 18.5, 5, 22, 5, 12, 10.5, 7.5, 21, 5, 15.5, 24]

# Valor de referência (hipótese nula: mediana igual a este valor)
valor_referencia = 10

diferencas = [x - valor_referencia for x in amostra]

estat_obs, p_valor = wilcoxon(diferencas, zero_method='wilcox', 
                         correction=False, alternative='less', method = 'auto')

if p_valor < alpha:
    print("Rejeitamos a hipótese nula")
else:
    print("Não rejeitamos a hipótese nula")

print(f"Valor-p: {p_valor:.4f}")

Não rejeitamos a hipótese nula
Valor-p: 0.8424


## 7) Teste Wilcoxon-Mann-Whitney (Mann-Whitney U)

**Teste para diferença de medianas de duas populações independentes**

Solomon e Coles, estudaram Ratos para saber se eles generalizam a aprendizagem por meio de imitação quando colocado sobre o novo desafio em uma nova situação.  Cinco ratos foram treinados para imitar ratos líderes em um labirinto. Eles foram treinados a seguir os líderes quando estivessem com fome a fim de obter alimento como recompensa. Depois, os cinco ratos foram transferidos para uma nova situação na qual a imitação dos Ratos líderes os tornaria capazes de evitar choques elétricos.  Seus comportamentos na situação de evitar choques foram comparados com 4 ratos de controle que não tiveram prévio treinamento para seguir líderes. 

A hipótese foi de que os cinco ratos que já tinham recebido o treinamento para imitar transfeririam este treinamento para a nova situação e, assim, atingiriam os critérios de aprendizagem na situação de evitar choques mais cedo do que os ratos controle. 

A comparação é feita em termos de quantos ensaios cada rato executou para chegar ao critério de 10 respostas corretas em 10 ensaios.
| Grupo experimental | Grupo controle |
|:------------------:|:--------------:|
|         110        |       78       |
|         70         |       64       |
|         53         |       75       |
|         51         |       45       |
|                    |       82       |

In [16]:
grupo_experimental = [110,70,53,51]
grupo_controle = [78,64,75,45,82]

In [17]:
alpha = 0.05

U, p_valor = mannwhitneyu(grupo_experimental, grupo_controle, 
                          use_continuity=True, alternative='greater', method='exact')
if p_valor < alpha:
    print("Rejeitamos a hipótese nula")
else:
    print("Não rejeitamos a hipótese nula")
print("Valor p:", p_valor.round(3))

Não rejeitamos a hipótese nula
Valor p: 0.635


#### Segundo exemplo: 

Vamos comparar as notas de física dos alunos de rede pública e privada. Nosso objetivo é entender se as escolas públicas e privadas possuem a mesma mediana de notas para a disciplina de Física. 

In [18]:
dados_alunos = pd.read_csv('Alunos_publica_privada.csv', sep= ';', decimal= ',')
dados_alunos.head()

,Sujeito,Genero,Escola,Posicao_Sala,Nota_Biol,Nota_Fis,Nota_Hist
0,1,M,Privada,Fundos,8.1,4.6,2.9
1,2,F,Privada,Fundos,5.3,4.9,6.4
2,3,M,Publica,Fundos,2.9,3.7,3.1
3,4,M,Publica,Fundos,3.7,3.8,4.9
4,5,F,Publica,Fundos,4.4,4.2,4.9


In [19]:
alpha = 0.05

privada = dados_alunos[dados_alunos.Escola == 'Privada']['Nota_Fis']
publica = dados_alunos[dados_alunos.Escola == 'Publica']['Nota_Fis']

U, p_valor = mannwhitneyu(privada, publica, use_continuity=True, 
                          alternative='two-sided', method='auto')

if p_valor < alpha:
    print("Rejeitamos a hipótese nula")
else:
    print("Não rejeitamos a hipótese nula")
print("Valor p:", p_valor.round(3))

Rejeitamos a hipótese nula
Valor p: 0.016


## 8) Teste Kruskall Wallis

**Teste para diferença de medianas de K populações independentes**

Adorno e colegas. Os escores estão apresentados abaixo. 
Verifique se há diferença na mediana dos grupos.

H0: Não há diferença na mediana do grau de autoritarismo dos 3 grupos.

H1: Pelo menos uma das medianas é diferentes. 




| Professores E.F. | Outros Professores | Administradores |
|:----------------:|:-----------:|:---------------:|
|        96        |      82     |       115       |
|        128       |     124     |       149       |
|        83        |     132     |       166       |
|        61        |     135     |       147       |
|        101       |     109     |       165          |

In [20]:
alpha = 0.05

prof_ef = [96,128,83,61,101]
prof_outros = [82,124,132,135,109]
admin = [115,149,166,147,165]

estat_obs, p_valor = kruskal(prof_ef, prof_outros, admin)

if p_valor < alpha:
    print("Rejeitamos a hipótese nula")
else:
    print("Não rejeitamos a hipótese nula")
    
print("Valor p:", p_valor.round(3))

Rejeitamos a hipótese nula
Valor p: 0.021


#### Utilizando o pacote pingouin

In [21]:
dados = pd.DataFrame({'prof_ef': prof_ef, 'prof_outros': prof_outros, 'admin': admin})

dados_pivotados = pd.melt(dados, var_name='Grupo', value_name='Grau')
dados_pivotados

,Grupo,Grau
0,prof_ef,96
1,prof_ef,128
2,prof_ef,83
3,prof_ef,61
4,prof_ef,101
5,prof_outros,82
6,prof_outros,124
7,prof_outros,132
8,prof_outros,135
9,prof_outros,109


In [22]:
Kruskall = pg.kruskal(data = dados_pivotados, dv='Grau', between='Grupo')
Kruskall

,Source,ddof1,H,p-unc
Kruskal,Grupo,2,7.74,0.020858


In [23]:
posthoc_dunn( dados_pivotados, val_col='Grau', group_col='Grupo', p_adjust = 'bonferroni')

,admin,prof_ef,prof_outros
admin,1.000000,0.017462,0.269058
prof_ef,0.017462,1.000000,0.866533
prof_outros,0.269058,0.866533,1.000000


## 9) Teste Q-Cochran

**Teste de diferença de proporções para K populações dependentes.** (Generalização do teste McNemar)

Deseja-se analisar a influência da atitude de um entrevistador sobre as respostas a uma pesquisa sobre opinião pública, dadas por donas de casa. Os entrevistadores são treinados de acordo com os seguintes métodos:

I – Demonstrando interesse, cordialidade, entusiasmo

II – Demonstrando formalismo, reserva e cortesia

III – Demonstrando modo abrupto, formalismo áspero. 


Cada entrevistador visita 18 casas (uma casa em cada bairro) e aplica um dos métodos de abordagem. 

Os resultados estão demonstrados na tabela ao lado. 
Obs: 0: não ; 1: sim

In [24]:
data = {
    'Bairro': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
    'Ent.1': [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
    'Ent.2': [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
    'Ent.3': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
        }

df = pd.DataFrame(data)

dados_pivotados = pd.melt(df, id_vars='Bairro', var_name='Grupos', value_name='Resposta')
dados_pivotados.head()

,Bairro,Grupos,Resposta
0,1,Ent.1,0
1,2,Ent.1,1
2,3,Ent.1,0
3,4,Ent.1,0
4,5,Ent.1,1


In [25]:
pg.cochran(data=dados_pivotados, subject='Bairro', dv='Resposta', within='Grupos')

,Source,dof,Q,p-unc
cochran,Grupos,2,16.666667,0.00024


In [26]:
posthoc_dunn(dados_pivotados, val_col='Resposta', group_col='Grupos', p_adjust = 'bonferroni')

,Ent.1,Ent.2,Ent.3
Ent.1,1.000000,1.000000,0.002785
Ent.2,1.000000,1.000000,0.002785
Ent.3,0.002785,0.002785,1.000000


## 9) Teste W-Friedman 

**ANOVA de dois fatores de Friedman por postos**

**Teste de diferença de medianas para K amostras dependentes**

Uma pesquisa foi delineada para avaliar a eficácia de um programa para a redução do estresse baseado em meditação, para pacientes com problemas de ansiedade. 

Foi medido o escore de Ansiedade da Escala Hamilton em quatro momentos diferentes: recrutamento (IR), pré-tratamento (Pré), pós-tratamento (Pós) e três meses após o pós-tratamento (3-M). 

O tratamento aparenta ter surtido efeito?

In [27]:
data = {
    'Sujeito': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
    'IR': [21, 30, 38, 43, 35, 40, 27, 18, 31, 21, 18, 28, 40, 35],
    'Pre': [21, 38, 19, 33, 34, 40, 15, 11, 42, 23, 24, 8, 37, 32],
    'Pos': [16, 10, 15, 30, 25, 31, 11, 4, 23, 21, 16, 5, 31, 12],
    '3M': [19, 21, 6, 24, 10, 30, 6, 7, 27, 17, 13, 2, 19, 21]
}

df = pd.DataFrame(data)

dados_pivotados = pd.melt(df, id_vars='Sujeito', var_name='Grupos', value_name='Escore')
dados_pivotados.head()

,Sujeito,Grupos,Escore
0,1,IR,21
1,2,IR,30
2,3,IR,38
3,4,IR,43
4,5,IR,35


In [28]:
pd.options.display.float_format = '{:.4f}'.format
pg.friedman(data=dados_pivotados, subject='Sujeito', dv='Escore', within='Grupos')

,Source,W,ddof1,Q,p-unc
Friedman,Grupos,0.8055,3,33.8321,0.0000


#### Testes post-hoc

In [29]:
# pingouin post-hoc 

pg.pairwise_tests(dv='Escore', within='Grupos', subject='Sujeito',data=dados_pivotados,
                  parametric= False, padjust='bonf', alpha = 0.05)

/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,Contrast,A,B,Paired,Parametric,W-val,alternative,p-unc,p-corr,p-adjust,hedges
0,Grupos,3M,IR,True,False,0.0000,two-sided,0.0001,0.0007,bonf,-1.6395
1,Grupos,3M,Pos,True,False,36.0000,two-sided,0.3258,1.0000,bonf,-0.2178
2,Grupos,3M,Pre,True,False,0.0000,two-sided,0.0001,0.0007,bonf,-1.0785
3,Grupos,IR,Pos,True,False,0.0000,two-sided,0.0016,0.0098,bonf,1.3698
4,Grupos,IR,Pre,True,False,23.0000,two-sided,0.2238,1.0000,bonf,0.3357
5,Grupos,Pos,Pre,True,False,0.0000,two-sided,0.0001,0.0007,bonf,-0.8632


In [30]:
# dunn post-hoc

posthoc_dunn(dados_pivotados, val_col='Escore', group_col='Grupos', p_adjust = 'bonferroni')

,3M,IR,Pos,Pre
3M,1.0000,0.0040,1.0000,0.0516
IR,0.0040,1.0000,0.0182,1.0000
Pos,1.0000,0.0182,1.0000,0.1727
Pre,0.0516,1.0000,0.1727,1.0000


In [31]:
# scipy post-hoc

scipost. posthoc_wilcoxon(dados_pivotados, group_col='Grupos', 
                          val_col='Escore', p_adjust='bonferroni', 
                          zero_method = 'wilcox', correction= True)

/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,IR,Pre,Pos,3M
IR,1.0000,1.0000,0.0098,0.0007
Pre,1.0000,1.0000,0.0007,0.0007
Pos,0.0098,0.0007,1.0000,1.0000
3M,0.0007,0.0007,1.0000,1.0000
